In [1]:
import re 
import string 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm
import json

from pandas.core.frame import DataFrame
from IPython.core.display import display, HTML


import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 


nlp = spacy.load("en_core_web_sm")

In [2]:
data= pd.read_csv("speeches.csv")
data

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8,10,11,12,15,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,speech_class,major_heading,minor_heading,oral_heading,year,hansard_membership_id,speakerid,person_id,speakername,url
0,uk.org.publicwhip/debate/1979-05-09a.1.1,"The House being met; and, it being the first d...",Unknown,NaN,NaN,NaN,1979-05-09,NaN,1,Procedural,Preamble,NaN,NaN,1979,NaN,NaN,NaN,Unknown,NaN
1,uk.org.publicwhip/debate/1979-05-09a.1.2,Several of the Members repaired to their Seats.,Unknown,NaN,NaN,NaN,1979-05-09,NaN,1,Procedural,Preamble,NaN,NaN,1979,NaN,NaN,NaN,Unknown,NaN
2,uk.org.publicwhip/debate/1979-05-09a.1.3,"JOHN PARKER, ESQUIRE, took the Chair, pursuant...",Unknown,NaN,NaN,NaN,1979-05-09,NaN,1,Procedural,Preamble,NaN,NaN,1979,NaN,NaN,NaN,Unknown,NaN
3,uk.org.publicwhip/debate/1979-05-09a.1.4,Message to attend the Lords Commissioners;,Unknown,NaN,NaN,NaN,1979-05-09,NaN,1,Procedural,Preamble,NaN,NaN,1979,NaN,NaN,NaN,Unknown,NaN
4,uk.org.publicwhip/debate/1979-05-09a.1.5,The House went; and a Commission having been r...,Unknown,NaN,NaN,NaN,1979-05-09,NaN,1,Procedural,Preamble,NaN,NaN,1979,NaN,NaN,NaN,Unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694370,uk.org.publicwhip/debate/2021-04-29a.519.2,I totally agree with everything that the right...,Lindsay Hoyle,Speaker,Chorley,467.0,2021-04-29,10:38:00,519,Speech,Point of Order,NaN,NaN,2021,NaN,NaN,uk.org.publicwhip/person/10295,Lindsay Hoyle,NaN
2694371,uk.org.publicwhip/debate/2021-04-29a.519.4,Message to attend the Lords Commissioners deli...,Unknown,NaN,NaN,NaN,2021-04-29,14:27:00,519,Procedural,Message to Attend the Lords Commissioners,NaN,NaN,2021,NaN,NaN,NaN,Unknown,NaN
2694372,uk.org.publicwhip/debate/2021-04-29a.519.6,I have to acquaint the House that the House ha...,Lindsay Hoyle,Speaker,Chorley,467.0,2021-04-29,14:27:00,519,Speech,Royal Assent,NaN,NaN,2021,NaN,NaN,uk.org.publicwhip/person/10295,Lindsay Hoyle,NaN
2694373,uk.org.publicwhip/debate/2021-04-29a.521.1,I have further to acquaint the House that the ...,Lindsay Hoyle,Speaker,Chorley,467.0,2021-04-29,14:27:00,521,Speech,Her Majesty’s Most Gracious Speech,NaN,NaN,2021,NaN,NaN,uk.org.publicwhip/person/10295,Lindsay Hoyle,NaN


In [36]:
print(data['speech'][2277224])

It is clear that exit checks, which were scrapped by the previous Labour Government, are a critical part of any competent immigration system. I know that progress has been made, but how sure is the Home Secretary that she will hit the target of 100% exit checks by March?


In [9]:
y = data['display_as']
print(len(set(y)))

2257


In [48]:
testdata = data[(data['year']>= 2015)] 
test = data[['id','speech','display_as','party','constituency']]
test

,id,speech,display_as,party,constituency
0,uk.org.publicwhip/debate/1979-05-09a.1.1,"The House being met; and, it being the first d...",Unknown,NaN,NaN
1,uk.org.publicwhip/debate/1979-05-09a.1.2,Several of the Members repaired to their Seats.,Unknown,NaN,NaN
2,uk.org.publicwhip/debate/1979-05-09a.1.3,"JOHN PARKER, ESQUIRE, took the Chair, pursuant...",Unknown,NaN,NaN
3,uk.org.publicwhip/debate/1979-05-09a.1.4,Message to attend the Lords Commissioners;,Unknown,NaN,NaN
4,uk.org.publicwhip/debate/1979-05-09a.1.5,The House went; and a Commission having been r...,Unknown,NaN,NaN
...,...,...,...,...,...
2694370,uk.org.publicwhip/debate/2021-04-29a.519.2,I totally agree with everything that the right...,Lindsay Hoyle,Speaker,Chorley
2694371,uk.org.publicwhip/debate/2021-04-29a.519.4,Message to attend the Lords Commissioners deli...,Unknown,NaN,NaN
2694372,uk.org.publicwhip/debate/2021-04-29a.519.6,I have to acquaint the House that the House ha...,Lindsay Hoyle,Speaker,Chorley
2694373,uk.org.publicwhip/debate/2021-04-29a.521.1,I have further to acquaint the House that the ...,Lindsay Hoyle,Speaker,Chorley


In [49]:
person = data[(data['display_as']== 'Lindsay Hoyle' )]
persondata = person[['id','speech','display_as','party','constituency']]

,id,speech,display_as,party,constituency
1214017,uk.org.publicwhip/debate/1997-05-21a.633.0,"I congratulate both you, Madam Speaker, and th...",Lindsay Hoyle,Labour,Chorley
1219452,uk.org.publicwhip/debate/1997-06-18a.327.4,May I congratulate my right hon. Friend on his...,Lindsay Hoyle,Labour,Chorley
1220368,uk.org.publicwhip/debate/1997-06-24a.669.2,To ask the Secretary of State for Health if he...,Lindsay Hoyle,Labour,Chorley
1220370,uk.org.publicwhip/debate/1997-06-24a.669.4,"I welcome that. I would, however, point out th...",Lindsay Hoyle,Labour,Chorley
1221144,uk.org.publicwhip/debate/1997-06-26a.979.3,To ask the Secretary of State for Education an...,Lindsay Hoyle,Labour,Chorley
...,...,...,...,...,...
2694368,uk.org.publicwhip/debate/2021-04-29a.518.6,"That brings me on to my final point, which is ...",Lindsay Hoyle,Speaker,Chorley
2694370,uk.org.publicwhip/debate/2021-04-29a.519.2,I totally agree with everything that the right...,Lindsay Hoyle,Speaker,Chorley
2694372,uk.org.publicwhip/debate/2021-04-29a.519.6,I have to acquaint the House that the House ha...,Lindsay Hoyle,Speaker,Chorley
2694373,uk.org.publicwhip/debate/2021-04-29a.521.1,I have further to acquaint the House that the ...,Lindsay Hoyle,Speaker,Chorley


In [51]:
# replace the pro
def replace(text):
    re = text.split()
    for i in range(len(re)):
        if re[i] == 'I':
            re[i] = 'Lindsay Hoyle'
    return ' '.join(re)
persondata['speech'] = persondata['speech'].apply(replace) 

<ipython-input-51-d8cf3cc871af>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persondata['speech'] = persondata['speech'].apply(replace)


In [54]:
ren

<ipython-input-54-1b84cf708a16>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persondata['named_ents'] = persondata['speech'].apply(extract_named_ents)


In [56]:
def extract_noun(text):
        return [(chunk.text) for chunk in nlp(text).noun_chunks]
persondata['Noun'] = persondata['speech'].apply(extract_noun)
persondata

<ipython-input-56-8b86bbb8a230>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persondata['Noun'] = persondata['speech'].apply(extract_noun)


,id,speech,display_as,party,constituency,named_ents,Noun
1214017,uk.org.publicwhip/debate/1997-05-21a.633.0,"Lindsay Hoyle congratulate both you, Madam Spe...",Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Council, ORG), (Lin...","[Lindsay Hoyle, both you, Madam Speaker, the P..."
1219452,uk.org.publicwhip/debate/1997-06-18a.327.4,May Lindsay Hoyle congratulate my right hon. F...,Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Europe, LOC), (Kohl...","[Lindsay Hoyle, my right hon, Friend, his succ..."
1220368,uk.org.publicwhip/debate/1997-06-24a.669.2,To ask the Secretary of State for Health if he...,Lindsay Hoyle,Labour,Chorley,"[(State, ORG), (Chorley and South, LOC), (3481...","[the Secretary, State, Health, he, the need, p..."
1220370,uk.org.publicwhip/debate/1997-06-24a.669.4,Lindsay Hoyle welcome that. Lindsay Hoyle woul...,Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Lindsay Hoyle, PERS...","[Lindsay Hoyle, Lindsay Hoyle, expectant mothe..."
1221144,uk.org.publicwhip/debate/1997-06-26a.979.3,To ask the Secretary of State for Education an...,Lindsay Hoyle,Labour,Chorley,"[(State, ORG), (4102, CARDINAL)]","[the Secretary, State, Education, Employment, ..."
...,...,...,...,...,...,...,...
2694368,uk.org.publicwhip/debate/2021-04-29a.518.6,"That brings me on to my final point, which is ...",Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (next Thursday, DATE...","[me, my final point, Lindsay Hoyle, next Thurs..."
2694370,uk.org.publicwhip/debate/2021-04-29a.519.2,Lindsay Hoyle totally agree with everything th...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (Gentleman, PERSON),...","[Lindsay Hoyle, everything, Gentleman, It, we,..."
2694372,uk.org.publicwhip/debate/2021-04-29a.519.6,Lindsay Hoyle have to acquaint the House that ...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (House, ORG), (House...","[Lindsay Hoyle, the House, the House, the Hous..."
2694373,uk.org.publicwhip/debate/2021-04-29a.521.1,Lindsay Hoyle have further to acquaint the Hou...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (House, ORG), (one, ...","[Lindsay Hoyle, the House, the Lord Privy Seal..."


In [62]:
def subtree_matcher(doc):
    subjpass = 0

    for i,tok in enumerate(doc):
        
    # find dependency tag that contains the text "subjpass"    
       if tok.dep_.find("subjpass") == True:
            subjpass = 1

    x = ''
    y = ''
    z = ''

  # if subjpass == 1 then sentence is passive
    if subjpass == 1:
        for i,tok in enumerate(doc):
            if tok.dep_.find("subj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    y = doc[i-1].text + ' '+tok.text
                else:
                    y=tok.text
                                   
            if(tok.dep_.find('ROOT')==0):
                z=tok.text
                
            if tok.dep_.endswith("obj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    x = doc[i-1].text + ' '+tok.text  
                else:
                    x = tok.text
  
  # if subjpass == 0 then sentence is not passive
    else:
        for i,tok in enumerate(doc):
            if tok.dep_.endswith("subj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    x = doc[i-1].text + ' '+tok.text 
                else:
                    x = tok.text
                    
            if(tok.dep_.find('ROOT')==0):
                z=tok.text

            if tok.dep_.endswith("obj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    y = doc[i-1].text + ' '+tok.text
                else:
                    y = tok.text

    return x,y,z

def get_relation(text):
    document = nlp(text)
    t = [sent.text for sent in document.sents]
    re =  []
    for i in t:
        i = nlp(i)
        re.append(subtree_matcher(i))
    return re

persondata['RE'] = persondata['speech'].apply(get_relation)
    
    

<ipython-input-62-d265aa096b5f>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persondata['RE'] = persondata['speech'].apply(get_relation)


In [63]:
def extract_verb(text):
    return [token for token in nlp(text)if token.pos_=='VERB']
persondata['Verb'] = persondata['speech'].apply(extract_verb)


<ipython-input-63-23fbf6087d63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persondata['Verb'] = persondata['speech'].apply(extract_verb)


In [64]:
persondata

,id,speech,display_as,party,constituency,named_ents,Noun,RE,Verb
1214017,uk.org.publicwhip/debate/1997-05-21a.633.0,"Lindsay Hoyle congratulate both you, Madam Spe...",Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Council, ORG), (Lin...","[Lindsay Hoyle, both you, Madam Speaker, the P...","[(Lindsay Hoyle, Council, congratulate), (Lind...","[congratulate, congratulate, hope, follow, pay..."
1219452,uk.org.publicwhip/debate/1997-06-18a.327.4,May Lindsay Hoyle congratulate my right hon. F...,Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Europe, LOC), (Kohl...","[Lindsay Hoyle, my right hon, Friend, his succ...","[(Lindsay Hoyle, right hon, congratulate), (, ...","[congratulate, have, depend, hope, did]"
1220368,uk.org.publicwhip/debate/1997-06-24a.669.2,To ask the Secretary of State for Health if he...,Lindsay Hoyle,Labour,Chorley,"[(State, ORG), (Chorley and South, LOC), (3481...","[the Secretary, State, Health, he, the need, p...","[(he, health district, ask), (, , 3481)]","[ask, assess]"
1220370,uk.org.publicwhip/debate/1997-06-24a.669.4,Lindsay Hoyle welcome that. Lindsay Hoyle woul...,Lindsay Hoyle,Labour,Chorley,"[(Lindsay Hoyle, PERSON), (Lindsay Hoyle, PERS...","[Lindsay Hoyle, Lindsay Hoyle, expectant mothe...","[(Lindsay Hoyle, that, welcome), (expectant mo...","[welcome, point, involved, have, travel, reach..."
1221144,uk.org.publicwhip/debate/1997-06-26a.979.3,To ask the Secretary of State for Education an...,Lindsay Hoyle,Labour,Chorley,"[(State, ORG), (4102, CARDINAL)]","[the Secretary, State, Education, Employment, ...","[(he, college status, ask), (, , 4102)]","[ask, plans, announce, concerning]"
...,...,...,...,...,...,...,...,...,...
2694368,uk.org.publicwhip/debate/2021-04-29a.518.6,"That brings me on to my final point, which is ...",Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (next Thursday, DATE...","[me, my final point, Lindsay Hoyle, next Thurs...","[(they, MP, brings), (Lindsay Hoyle, complaint...","[brings, is, say, be, is, remind, going, ensur..."
2694370,uk.org.publicwhip/debate/2021-04-29a.519.2,Lindsay Hoyle totally agree with everything th...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (Gentleman, PERSON),...","[Lindsay Hoyle, everything, Gentleman, It, we,...","[(that, everything, agree), (Gentleman, , said...","[agree, said, have, know, need, answered, have..."
2694372,uk.org.publicwhip/debate/2021-04-29a.519.6,Lindsay Hoyle have to acquaint the House that ...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (House, ORG), (House...","[Lindsay Hoyle, the House, the House, the Hous...","[(following Acts, Commission, have), (, Educat...","[have, acquaint, been, read, authorising, foll..."
2694373,uk.org.publicwhip/debate/2021-04-29a.521.1,Lindsay Hoyle have further to acquaint the Hou...,Lindsay Hoyle,Speaker,Chorley,"[(Lindsay Hoyle, PERSON), (House, ORG), (one, ...","[Lindsay Hoyle, the House, the Lord Privy Seal...","[(Privy Seal, Command, have), (House, terms, o...","[have, acquaint, delivered, obtained, directed..."
